In [1]:
import pandas as pd
data = pd.read_csv('preprocessed_data_with_numerical.csv')

In [2]:
data.fillna('', inplace=True)

In [3]:
data['full_text'] = data['title'] + " " + data['location']  + " " + data['department']  + " " + data['company_profile']  + " " + data['description']  + " " + data['requirements']  + " "  + data['benefits'] + data['industry']  + " " + data['function']

In [4]:
data

,title,location,department,company_profile,description,requirements,benefits,telecommuting,has_company_logo,has_questions,employment_type,required_experience,required_education,industry,function,fraudulent,in_balanced_dataset,missing company profile,missing company information,full_text
0,marketing intern,usa ny new york,marketing,food52 created groundbreaking award winning co...,food52 fast growing james beard award winning ...,experience content management system major plu...,,0,1,0,2,4,9,,marketing,0,0,1,3,marketing intern usa ny new york marketing foo...
1,customer service cloud video production,nz auckland,success,90 second world cloud video production service...,organised focused vibrant awesome passion cust...,expect key responsibility communicate client 9...,get u part 90 second team gain experience work...,0,1,0,1,6,9,marketing advertising,customer service,0,0,1,3,customer service cloud video production nz auc...
2,commissioning machinery assistant cma,usa ia wever,,valor service provides workforce solution meet...,client located houston actively seeking experi...,implement pre commissioning commissioning proc...,,0,1,0,2,6,9,,,0,0,1,3,commissioning machinery assistant cma usa ia w...
3,account executive washington dc,usa dc washington,sale,passion improving quality life geography heart...,company esri environmental system research ins...,education bachelor master gi business administ...,culture anything corporate collaborative creat...,0,1,0,1,5,1,computer software,sale,0,0,1,3,account executive washington dc usa dc washing...
4,bill review manager,usa fl fort worth,,spotsource solution llc global human capital m...,job title itemization review manager location ...,qualification rn license state texas diploma b...,full benefit offered,0,1,1,1,5,1,hospital health care,health care provider,0,0,1,3,bill review manager usa fl fort worth spotsou...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17875,account director distribution,ca toronto,sale,vend looking awesome new talent come join u wo...,case first time visited website vend award win...,ace role eat comprehensive statement work brea...,expect u open culture openly share result inpu...,0,1,1,1,5,9,computer software,sale,0,0,1,3,account director distribution ca toronto sale ...
17876,payroll accountant,usa pa philadelphia,accounting,weblinc e commerce platform service provider f...,payroll accountant focus primarily payroll fun...,b b accounting desire fun love genuine passion...,health wellness medical plan prescription drug...,0,1,1,1,5,1,internet,accounting auditing,0,0,1,3,payroll accountant usa pa philadelphia account...
17877,project cost control staff engineer cost contr...,usa tx houston,,provide full time permanent position many medi...,experienced project cost control staff enginee...,least 12 year professional experience ability ...,,0,0,0,1,6,9,,,0,0,1,2,project cost control staff engineer cost contr...
17878,graphic designer,ng la lagos,,,nemsia studio looking experienced visual graph...,1 must fluent latest version corel adobe cc es...,competitive salary compensation based experien...,0,0,1,0,6,6,graphic design,design,0,0,0,0,graphic designer ng la lagos nemsia studio l...


In [5]:
X = data[['full_text', 'has_questions', 'required_experience', 'required_education', 'missing company information']]
# X = data['full_text']
y = data['fraudulent']
from sklearn.model_selection import train_test_split
X_train, X_test,y_train , y_test = train_test_split(X, y, test_size=0.3, random_state=0)

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.preprocessing import StandardScaler
from scipy.sparse import hstack
# from sklearn.pipeline import FeatureUnion, Pipeline
import numpy as np
from sklearn.decomposition import PCA
from sklearn.metrics import f1_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from xgboost import XGBClassifier

<h1>Text Only Models</h1>

TFIDF Vectorizer

In [7]:
tfidf_vectorizer = TfidfVectorizer(stop_words='english')
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train['full_text'])
X_test_tfidf = tfidf_vectorizer.transform(X_test['full_text'])

In [8]:
def train_and_evaluate_model(model, X_train, y_train, X_test, y_test):
    # Train the model
    model.fit(X_train, y_train)
    
    # Make predictions on the test set
    y_pred = model.predict(X_test)
    
    # Evaluate F1 score
    f1 = f1_score(y_test, y_pred)
    
    return f1

# Step 4: Logistic Regression
logreg_model = LogisticRegression()
logreg_f1 = train_and_evaluate_model(logreg_model, X_train_tfidf, y_train, X_test_tfidf, y_test)
print(f"Logistic Regression F1 Score: {logreg_f1}")

# Step 5: Support Vector Machine (SVM)
svm_model = SVC()
svm_f1 = train_and_evaluate_model(svm_model, X_train_tfidf, y_train, X_test_tfidf, y_test)
print(f"SVM F1 Score: {svm_f1}")

# Step 6: Random Forest
rf_model = RandomForestClassifier()
rf_f1 = train_and_evaluate_model(rf_model, X_train_tfidf, y_train, X_test_tfidf, y_test)
print(f"Random Forest F1 Score: {rf_f1}")

xgb_model = XGBClassifier()
xgb_f1 = train_and_evaluate_model(xgb_model, X_train_tfidf, y_train, X_test_tfidf, y_test)
print(f"XGBoost F1 Score: {xgb_f1}")

Logistic Regression F1 Score: 0.625
SVM F1 Score: 0.8153846153846154
Random Forest F1 Score: 0.793733681462141
XGBoost F1 Score: 0.8226600985221675


Word2Vec

In [11]:
from gensim.models import Word2Vec
from gensim.models.doc2vec import TaggedDocument, Doc2Vec

# Step 2: Tokenize the text
tokenized_text = X_train['full_text'].apply(lambda x: x.split())

# Step 3: Train Word2Vec model
word2vec_model = Word2Vec(sentences=tokenized_text, vector_size=100, window=5, min_count=1, workers=4)

# Step 4: Convert words to vectors
def get_vector(word_list, model):
    valid_words = [word for word in word_list if word in model.wv]
    if not valid_words:
        # If no valid words, return a vector of zeros or handle as needed
        return np.zeros(model.vector_size)
    return np.mean([model.wv[word] for word in valid_words], axis=0)

X_train_word2vec = tokenized_text.apply(lambda x: get_vector(x, word2vec_model))
X_test_word2vec = X_test['full_text'].apply(lambda x: get_vector(x.split(), word2vec_model))

In [12]:
# Step 5: Train models and evaluate F1 score
def train_and_evaluate_model(model, X_train, y_train, X_test, y_test):
    # Train the model
    model.fit(X_train.to_list(), y_train)
    
    # Make predictions on the test set
    y_pred = model.predict(X_test.to_list())
    
    # Evaluate F1 score
    f1 = f1_score(y_test, y_pred)
    
    return f1

# Step 6: Logistic Regression
logreg_model = LogisticRegression()
logreg_f1 = train_and_evaluate_model(logreg_model, X_train_word2vec, y_train, X_test_word2vec, y_test)
print(f"Logistic Regression F1 Score: {logreg_f1}")

# Step 7: Support Vector Machine (SVM)
svm_model = SVC()
svm_f1 = train_and_evaluate_model(svm_model, X_train_word2vec, y_train, X_test_word2vec, y_test)
print(f"SVM F1 Score: {svm_f1}")

# Step 8: Random Forest
rf_model = RandomForestClassifier()
rf_f1 = train_and_evaluate_model(rf_model, X_train_word2vec, y_train, X_test_word2vec, y_test)
print(f"Random Forest F1 Score: {rf_f1}")

xgb_model = XGBClassifier()
xgb_f1 = train_and_evaluate_model(xgb_model, X_train_word2vec, y_train, X_test_word2vec, y_test)
print(f"XGBoost F1 Score: {xgb_f1}")

/Users/natthaphonkanthawang/opt/anaconda3/envs/myenv/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Logistic Regression F1 Score: 0.4222873900293255
SVM F1 Score: 0.5123456790123456
Random Forest F1 Score: 0.5928143712574849
XGBoost F1 Score: 0.7037037037037037


CountVectorizer

In [13]:
count_vectorizer = CountVectorizer()
X_train_cv = count_vectorizer.fit_transform(X_train['full_text'])
X_test_cv = count_vectorizer.transform(X_test['full_text'])

In [14]:
def train_and_evaluate_model(model, X_train, y_train, X_test, y_test):
    # Train the model
    model.fit(X_train, y_train)
    
    # Make predictions on the test set
    y_pred = model.predict(X_test)
    
    # Evaluate F1 score
    f1 = f1_score(y_test, y_pred)
    
    return f1

# Step 4: Logistic Regression
logreg_model = LogisticRegression()
logreg_f1 = train_and_evaluate_model(logreg_model, X_train_cv, y_train, X_test_cv, y_test)
print(f"Logistic Regression F1 Score: {logreg_f1}")

# Step 5: Support Vector Machine (SVM)
svm_model = SVC()
svm_f1 = train_and_evaluate_model(svm_model, X_train_cv, y_train, X_test_cv, y_test)
print(f"SVM F1 Score: {svm_f1}")

# Step 6: Random Forest
rf_model = RandomForestClassifier()
rf_f1 = train_and_evaluate_model(rf_model, X_train_cv, y_train, X_test_cv, y_test)
print(f"Random Forest F1 Score: {rf_f1}")

xgb_model = XGBClassifier()
xgb_f1 = train_and_evaluate_model(xgb_model, X_train_cv, y_train, X_test_cv, y_test)
print(f"XGBoost F1 Score: {xgb_f1}")

/Users/natthaphonkanthawang/opt/anaconda3/envs/myenv/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Logistic Regression F1 Score: 0.8009153318077803
SVM F1 Score: 0.7272727272727272
Random Forest F1 Score: 0.7905759162303664
XGBoost F1 Score: 0.8382352941176471


<h3>Using text features only</h3>

|  | Logistic Regression | SVM | Random Forest | XGBoost | Average |
|----------|----------|----------|----------|----------|----------|
| TFIDF | 0.625 | 0.815 | 0.787 | 0.827 | 0.764 |
| Word2Vec | 0.401 | 0.509 | 0.571 | 0.699 | 0.545 |
| CountVectorizer | 0.801 | 0.727 | 0.794 | 0.838 | 0.790 |


Based on the results above, the best word embedding method to use is CountVectorizer that obtained the highest average F1 score of 0.790 across all models. Hence, we will be using CountVectorizer moving forward.

 Long Short Term Memory (LSTM)

In [35]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Embedding, LSTM, Dense,Bidirectional
from tensorflow.keras.layers import Dropout
from tensorflow.keras.preprocessing.text import one_hot

In [44]:
voc_size=10000
corpus = data["full_text"]
onehot_repr=[one_hot(words,voc_size)for words in corpus] 
onehot_repr[1]


[5567,
 7249,
 629,
 8899,
 2221,
 4435,
 7360,
 3650,
 5537,
 4648,
 9627,
 629,
 8899,
 2221,
 7249,
 5537,
 4648,
 9627,
 629,
 8899,
 2221,
 7249,
 711,
 1264,
 8550,
 5410,
 1845,
 2711,
 2290,
 8899,
 7113,
 3582,
 1939,
 8347,
 9627,
 5537,
 4648,
 3992,
 8899,
 2221,
 3874,
 7916,
 4984,
 8973,
 629,
 7998,
 543,
 6898,
 5537,
 3443,
 7617,
 5537,
 4648,
 8427,
 4834,
 6257,
 7240,
 5402,
 8521,
 5088,
 3032,
 8899,
 2221,
 4915,
 3508,
 3929,
 7199,
 8899,
 6230,
 6071,
 2290,
 9897,
 2031,
 7856,
 6662,
 3850,
 5366,
 7996,
 8899,
 516,
 6511,
 6482,
 4984,
 2417,
 2221,
 3650,
 2898,
 1512,
 6482,
 5537,
 4648,
 1292,
 6392,
 3650,
 6749,
 5537,
 4648,
 1939,
 7412,
 9114,
 5366,
 8899,
 5424,
 6482,
 6713,
 7856,
 1264,
 5501,
 9627,
 7703,
 5501,
 321,
 7577,
 7723,
 9426,
 7070,
 5346,
 7360,
 8230,
 8280,
 6324,
 2492,
 6898,
 5537,
 3443,
 7617,
 6898,
 5537,
 3443,
 6639,
 6898,
 5537,
 3443,
 3450,
 3178,
 485,
 7212,
 9857,
 5502,
 5567,
 7249,
 4441,
 6181,
 6929,
 

In [45]:
sent_length=50
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[7742  178 4774 ... 4210 6447 1565]
 [2686 6692 7834 ... 4395 5567 7249]
 [7260 3753 6451 ... 5728 6230 4186]
 ...
 [7897 6156 8448 ... 7030 3443 3657]
 [9064 5609 7278 ... 3266 5609 5609]
 [7648 9945 3876 ... 7982 2030 1795]]


In [57]:
embedding_vector_features=50
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(Bidirectional(LSTM(100))) ##Just add bidirectional!!, except it would just behave as normal LSTM Model
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential_19"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 50, 50)            500000    
                                                                 
 bidirectional_16 (Bidirecti  (None, 200)              120800    
 onal)                                                           
                                                                 
 dropout_15 (Dropout)        (None, 200)               0         
                                                                 
 dense_16 (Dense)            (None, 1)                 201       
                                                                 
Total params: 621,001
Trainable params: 621,001
Non-trainable params: 0
_________________________________________________________________
None


In [59]:
X_train_lstm, X_test_lstm, y_train_lstm, y_test_lstm = train_test_split(embedded_docs, y, test_size=0.3, random_state=0)

In [60]:
model.fit(X_train_lstm, y_train_lstm, epochs=10, batch_size=32, validation_data=(X_test_lstm, y_test_lstm))

Epoch 1/10
392/392 [==============================] - 26s 65ms/step - loss: 0.0065 - accuracy: 0.9978 - val_loss: 0.1136 - val_accuracy: 0.9754
Epoch 2/10
392/392 [==============================] - 24s 60ms/step - loss: 0.0030 - accuracy: 0.9992 - val_loss: 0.1333 - val_accuracy: 0.9789
Epoch 3/10
392/392 [==============================] - 24s 60ms/step - loss: 0.0075 - accuracy: 0.9982 - val_loss: 0.1034 - val_accuracy: 0.9756
Epoch 4/10
392/392 [==============================] - 25s 63ms/step - loss: 0.0065 - accuracy: 0.9986 - val_loss: 0.1350 - val_accuracy: 0.9808
Epoch 5/10
392/392 [==============================] - 24s 62ms/step - loss: 0.0012 - accuracy: 0.9998 - val_loss: 0.1441 - val_accuracy: 0.9812
Epoch 6/10
392/392 [==============================] - 24s 61ms/step - loss: 8.4761e-04 - accuracy: 0.9998 - val_loss: 0.1464 - val_accuracy: 0.9815
Epoch 7/10
392/392 [==============================] - 30s 78ms/step - loss: 5.7618e-04 - accuracy: 0.9998 - val_loss: 0.1435 - val_a

In [61]:
# Evaluate the model on the test set
loss, accuracy = model.evaluate(X_test_lstm, y_test_lstm)

print(f"Test Accuracy: {accuracy}")
print(f"Test Loss: {loss}")

168/168 [==============================] - 5s 27ms/step - loss: 0.1787 - accuracy: 0.9810
Test Accuracy: 0.9809843301773071
Test Loss: 0.17866438627243042


In [62]:
y_pred = model.predict(X_test_lstm)

168/168 [==============================] - 5s 22ms/step


In [66]:
from sklearn.metrics import classification_report
y_pred_binary = (y_pred > 0.5).astype('int32')  
# Get classification report
report = classification_report(y_test_lstm, y_pred.round(),target_names = ['0','1'])
print("Classification Report:")
print(report)

Classification Report:
              precision    recall  f1-score   support

           0       0.98      1.00      0.99      5137
           1       0.89      0.63      0.74       227

    accuracy                           0.98      5364
   macro avg       0.94      0.81      0.86      5364
weighted avg       0.98      0.98      0.98      5364



In [65]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test,y_pred_binary)

array([[5120,   17],
       [  85,  142]])

In [67]:
from sklearn.metrics import f1_score
f1_test = f1_score(y_test_lstm, y_pred_binary )
print(f'LSTM F1 score: {f1_test}')

LSTM F1 score: 0.7357512953367876


<h1>Combined text and numeric</h1>

tfidf and numeric

In [15]:
numeric_features = X_train[['has_company_logo', 'has_questions', 'employment_type', 'required_experience', 'required_education']]
combined_features = np.hstack([
    StandardScaler().fit_transform(numeric_features),
    X_train_tfidf.toarray()])
X_test_numeric_features = X_test[['has_company_logo', 'has_questions', 'employment_type', 'required_experience', 'required_education']]
X_test_combined_features = np.hstack([
    StandardScaler().fit_transform(X_test_numeric_features),
    X_test_tfidf.toarray()])

def train_and_evaluate_model(model, X_train, y_train, X_test, y_test):
    # Train the model
    model.fit(X_train, y_train)
    
    # Make predictions on the test set
    y_pred = model.predict(X_test)
    
    # Evaluate F1 score
    f1 = f1_score(y_test, y_pred)
    
    return f1
logreg_model = LogisticRegression()
logreg_f1 = train_and_evaluate_model(logreg_model, combined_features, y_train, X_test_combined_features, y_test)
print(f"Logistic Regression F1 Score: {logreg_f1}")

# Step 5: Support Vector Machine (SVM)
svm_model = SVC()
svm_f1 = train_and_evaluate_model(svm_model, combined_features, y_train, X_test_combined_features, y_test)
print(f"SVM F1 Score: {svm_f1}")

# Step 6: Random Forest
rf_model = RandomForestClassifier()
rf_f1 = train_and_evaluate_model(rf_model, combined_features, y_train, X_test_combined_features, y_test)
print(f"Random Forest F1 Score: {rf_f1}")

KeyError: "['has_company_logo', 'employment_type'] not in index"

In [ ]:
train_word2vec_array = np.vstack(X_train_word2vec.apply(lambda x: x.flatten()).to_numpy())
test_word2vec_array = np.vstack(X_test_word2vec.apply(lambda x: x.flatten()).to_numpy())
combined_features = np.hstack([
    StandardScaler().fit_transform(numeric_features),
    train_word2vec_array])
X_test_numeric_features = X_test[['has_company_logo', 'has_questions', 'employment_type', 'required_experience', 'required_education']]
X_test_combined_features = np.hstack([
    StandardScaler().fit_transform(X_test_numeric_features),
    test_word2vec_array])

def train_and_evaluate_model(model, X_train, y_train, X_test, y_test):
    # Train the model
    model.fit(X_train, y_train)
    
    # Make predictions on the test set
    y_pred = model.predict(X_test)
    
    # Evaluate F1 score
    f1 = f1_score(y_test, y_pred)
    
    return f1
logreg_model = LogisticRegression()
logreg_f1 = train_and_evaluate_model(logreg_model, combined_features, y_train, X_test_combined_features, y_test)
print(f"Logistic Regression F1 Score: {logreg_f1}")

# Step 5: Support Vector Machine (SVM)
svm_model = SVC()
svm_f1 = train_and_evaluate_model(svm_model, combined_features, y_train, X_test_combined_features, y_test)
print(f"SVM F1 Score: {svm_f1}")

# Step 6: Random Forest
rf_model = RandomForestClassifier()
rf_f1 = train_and_evaluate_model(rf_model, combined_features, y_train, X_test_combined_features, y_test)
print(f"Random Forest F1 Score: {rf_f1}")

In [ ]:
numeric_features = X_train[['has_questions', 'required_experience', 'required_education', 'missing company information']]
combined_features = np.hstack([
    StandardScaler().fit_transform(numeric_features),
    X_train_cv.toarray()])
X_test_numeric_features = X_test[['has_questions', 'required_experience', 'required_education', 'missing company information']]
X_test_combined_features = np.hstack([
    StandardScaler().fit_transform(X_test_numeric_features),
    X_test_cv.toarray()])

def train_and_evaluate_model(model, X_train, y_train, X_test, y_test):
    # Train the model
    model.fit(X_train, y_train)
    
    # Make predictions on the test set
    y_pred = model.predict(X_test)
    
    # Evaluate F1 score
    f1 = f1_score(y_test, y_pred)
    
    return f1
logreg_model = LogisticRegression()
logreg_f1 = train_and_evaluate_model(logreg_model, combined_features, y_train, X_test_combined_features, y_test)
print(f"Logistic Regression F1 Score: {logreg_f1}")

# Step 5: Support Vector Machine (SVM)
svm_model = SVC()
svm_f1 = train_and_evaluate_model(svm_model, combined_features, y_train, X_test_combined_features, y_test)
print(f"SVM F1 Score: {svm_f1}")

# Step 6: Random Forest
rf_model = RandomForestClassifier()
rf_f1 = train_and_evaluate_model(rf_model, combined_features, y_train, X_test_combined_features, y_test)
print(f"Random Forest F1 Score: {rf_f1}")

xgb_model = XGBClassifier()
xgb_f1 = train_and_evaluate_model(xgb_model, combined_features, y_train, X_test_combined_features, y_test)
print(f"XGBoost F1 Score: {xgb_f1}")

<h3>Combined features</h3>

|  | Logistic Regression | SVM | Random Forest | XGBoost |
|----------|----------|----------|----------|----------|
| Text Only | 0.801 | 0.727 | 0.794 | 0.838 |
| Text and Numeric | 0.823 | 0.729 | 0.791 | 0.846 |

In [ ]:
# neural network

import tensorflow as tf
from tensorflow import keras
from sklearn.metrics import f1_score

# Define and compile the neural network model
model = keras.Sequential([
    keras.layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy')

# Train the model on the training data
model.fit(X_train, y_train, epochs=10, batch_size=32)

# Make predictions on the validation data
y_pred_valid = model.predict(X_val)
y_pred_valid = (y_pred_valid > 0.5).astype(int)  # Convert probabilities to binary labels

# Calculate the F1 score on the validation set
f1_valid = f1_score(y_val, y_pred_valid)

print("Validation Set F1 Score:")
print(f"F1 Score: {f1_valid}")

# Make predictions on the test data
y_pred_test = model.predict(X_test)
y_pred_test = (y_pred_test > 0.5).astype(int)  # Convert probabilities to binary labels

# Calculate the F1 score on the test set
f1_test = f1_score(y_test, y_pred_test)

print("Test Set F1 Score:")
print(f"F1 Score: {f1_test}")